In [1]:
###########
# IMPORTS #
###########
import tensorflow as tf

#Abstractive Summarizer Imports
import datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import BertTokenizer, EncoderDecoderModel

#Extractive Summarizer Imports
from tqdm import tqdm_pandas
from summarizer import Summarizer

#Cosine Similarity Imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

#Utility Imports
from functools import reduce
from operator import add
import pandas as pd
import numpy as np

#Dataset Library Imports
from sklearn.datasets import fetch_20newsgroups

In [2]:
%%capture
###############
# GLOBAL VARS #
###############

vectorizer = TfidfVectorizer()
tokenizer = AutoTokenizer.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")  
abstractive_summarizer_model = AutoModelForSeq2SeqLM.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")
extractive_summarizer_model = Summarizer()

In [ ]:
########
# DATA #
########

#CNN Dailymail dataset, initially used for testing summarizers
test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test")
twenty_news_dataset = fetch_20newsgroups()

#Toy data for testing the pipeline
covid_test_data = pd.read_csv("/home/jupyter/266/266_final/nyt_data_collection/toy_data/fp_covid_articles.csv")
golf_test_data = pd.read_csv("/home/jupyter/266/266_final/nyt_data_collection/toy_data/golf_articles.csv")

In [ ]:
####################
# HELPER FUNCTIONS #
####################

# MISC Helpers
divString = lambda size, char = "#": reduce(add, [char for i in range(size)])
flatten = lambda lst: [i for sublst in lst for i in sublst]


#Batch Summary Generation and Batch Metrics
def generate_summary(batch):
    """This function computes a summary for a given article from the Dataset object
    batch
    Params:
    batch: an article from the given Dataset object."""
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch["pred"] = output_str
    return batch


def compute_metrics(batch, batch_size=16, metric_name="rouge"):
    """This function computes the rouge or bleu scores for predicted summaries
    Params:
    batch: A Dataset object which contains the articles at the specified indices
    Use the select method for this function call. 
    Example format: Dataset.select([list of indices to select from the original dataset])
    metric_name: The prefered evaluation metric to use"""
    
    metric = datasets.load_metric(metric_name)
    results = batch.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["article"])
    summary_pred = results["pred"]
    label_ref = results["highlights"]
    if metric_name == "rouge":
        output = metric.compute(predictions=summary_pred, references=label_ref, rouge_types=["rouge2"])["rouge2"].mid
        print("\n" + "ROUGE SCORE:")
        return output
    else:
        # Else compute bleu score with metric name "sacrebleu"
        all_bleu_scores = []
        for i in range(len(batch)):
            output = metric.compute(predictions= [summary_pred[i]], references= [[label_ref[i]]])
            all_bleu_scores.append(output)
            print("\n\n")
            print(divString(100))
            print("\n\n" + "Summary prediction: " + "\n\n", summary_pred[i])
            print("\n\n" + "Reference Label: " + "\n\n", label_ref[i])
            print("\n\n" + "BLEU SCORE:" + "\n\n", output)
            print("\n")
        return all_bleu_scores
    

#Raw Text Summarization
def generate_abstractive_summary(raw_string, model = abstractive_summarizer_model):
    """This function produces an abstractive summary for a given article"
    Params:
    raw_string: an article string.
    model: An abstractive summarizer model"""
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(raw_string, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return output_str[0]


def generate_extractive_summary(raw_string, model = extractive_summarizer_model, min_summary_length = 50):
    """This function produces an extractive summary for a given article"
    Params:
    raw_string: an article string.
    model: An extractive summarizer model"""
    output_str = model(raw_string, min_length = min_summary_length)
    return output_str
    
    
#Search and Subset Dataset
def search_and_subset_data(df, keyword, column = "first_paragraph"):
    """
    This function takes in a dataframe, keyword to search, and an optional column to search through and returns a
    subset of the data as a pandas dataframe, with entries that contain the searched keyword.
    Params:
    df: Dataframe
    keyword: A keyword to search
    column: Optional that takes either 'first_paragraph' or 'keywords'
    """
    df = df.sort_values(by='date', ascending = False).reset_index().drop("index", axis = 1) # Ordering all documents chronologically so that indices don't need reodered when combining similar documents
    df = df.dropna(subset=[column])
    subset = df[df[column].str.lower().str.contains(keyword.strip().lower())]
    return subset


def select_random_document(df):
    """
    This function selects a single random row from a dataframe. This is used as a default for initial document selection at the beginning of the pipeline.
    df: A pandas dataframe
    """
    return df.sample()



# Similarity Clustering and Aggregate Document Synthesis
def compute_cosine_similarities(document, corpus, vectorizer = vectorizer):
    """
    This function computes the cosine similarity between a document and a specified corpus of documents.
    document: A string of text.
    corpus: An array of documents.
    vectorizer: A TfidfVectorizer() object. (Default: initialized in the constants cell)
    """
    vectorized_corpus = vectorizer.fit_transform(corpus)
    vectorized_document = vectorizer.transform([document])
    return linear_kernel(vectorized_document, vectorized_corpus).flatten()


def get_related_docs_indices(cos_similarities_array, n_docs=5):
    """
    This function returns the document indices with the highest cosine similarity.
    cos_similarities_array: An array of the computed cosine similarities for the whole corpus.
    n_docs: The number of highest scoring documents to return. (e.g. n_docs=5 returns the top 5 highest scoring documents)
    """
    cos_similarities_array = np.array([i for i in cos_similarities_array if i < 0.999999999]) # This eliminates the case where the most similar document is the document itself, which has a similarity of 1.0
    return sorted(cos_similarities_array.argsort()[:-(n_docs + 1):-1])
    

def get_top_similarities(cos_similarities_array, n_docs=5):
    """
    This function returns the highest document cosine similarity scores.
    cos_similarities_array: An array of the computed cosine similarities for the whole corpus.
    n_docs: The number of highest cosine scores to return. (e.g. n_docs=5 returns the top 5 highest cosine similarity scores)
    """
    related_indices = get_related_docs_indices(cos_similarities_array, n_docs)
    return cos_similarities_array[related_indices]


def concatenate_related_docs(corpus, related_docs_indices):
    docs = [corpus[i] for i in related_docs_indices]
    return " ".join(docs)
    

# Display Functions
def show_related_docs(document, corpus, related_docs_indices):
    """
    This function displays the seed document, selected similar documents, and the concatenated aggregate of the similar documents.
    """
    aggregate_doc = concatenate_related_docs(corpus, related_docs_indices)
    print("\n" + "SELECTED DOCUMENT: " + "\n")
    print(document)
    print("\n")
    print(divString(100))
    print("\n")
    print("SIMILAR DOCUMENTS: " + "\n")
    for i in related_docs_indices:
        print(corpus[i], "\n")
        print(divString(100, char = "~") + "\n")
    print(divString(100))
    print("\n")
    print("AGGREGATE DOCUMENT: " + "\n")
    print(aggregate_doc + "\n")
    print(divString(100))


#Pipeline Iteration
def iterate_pipeline(initial_document, corpus, num_iter = 5, aggregate_doc_variant = "abstractive_secondary"):
    """
    This function begins with an initial document. From there the initial document is clustered with similar documents, 
    """
    document = initial_document
    for i in range(num_iter):
        cosine_similarities = compute_cosine_similarities(document, corpus)
        related_docs_indices = get_related_docs_indices(cosine_similarities)
        aggregate_document = concatenate_related_docs(corpus, related_docs_indices)
        if aggregate_doc_variant == "abstractive_primary":
            document = generate_abstractive_summary(aggregate_document, model = abstractive_summarizer_model)
        if aggregate_doc_variant == "abstractive_secondary":
            extractive_summary_primary = generate_extractive_summary(aggregate_document, min_summary_length=100)
            document = generate_abstractive_summary(extractive_summary_primary, model = abstractive_summarizer_model)
        if aggregate_doc_variant == "extractive_primary":
            document = generate_extractive_summary(aggregate_document, min_summary_length=100)
    print("Seed Document:" + "\n")
    print(initial_document, "\n")
    print(divString(100) + "\n")
    print("Resulting Document:" + "\n")
    print(document, "\n")
    return document

In [ ]:
# Helper Function Unit Tests

# generate_summary(test_data[0])
# compute_metrics(test_data.select([1,2]), metric_name = "rouge")
# compute_metrics(test_data.select([1, 2]), metric_name = "sacrebleu")
# generate_extractive_summary(test_data[0]["article"])
# generate_extractive_summary(test_data[0]["article"], min_summary_length=10)

### Baseline Algorithm Walkthrough
1. Select a group/subset of articles (Order the subset chronologically so that the dataframe indices are chronological)
2. Select a single article from the subset produced in step 1. (Baseline selection will be random)
3. Perform cosine similarity between the single/selected article and the entire subset of articles produced in step 1.
4. Select the top most similar indices and their associated articles.
5. Summarize them.

In [ ]:
############
# PIPELINE #
############

# Document Selection
search_results_df = search_and_subset_data(covid_test_data, "death") # a df containing entries that have death in the first paragraph
corpus = search_results_df.first_paragraph.to_list()  # Converting to a list to vectorize the entries
document = select_random_document(search_results_df).first_paragraph.values[0] # Selected a random row from the search results dataframe and extracted the first paragraph text to serve as our document

# Similarity Clustering and Aggregate Document Synthesis
cosine_similarities = compute_cosine_similarities(document, corpus) # The cosine similarities between the target document and all documents contained in the corpus
related_docs_indices = get_related_docs_indices(cosine_similarities) # The indices of the most related docs
aggregate_document = concatenate_related_docs(corpus, related_docs_indices) # The resulting document that is produced by concatenating all of the most similar documents

# Summarization
extractive_summary_primary = generate_extractive_summary(aggregate_document, min_summary_length=100)
abstractive_summary_primary = generate_abstractive_summary(aggregate_document, model = abstractive_summarizer_model)
abstractive_summary_secondary = generate_abstractive_summary(extractive_summary_primary, model = abstractive_summarizer_model) #Secondary summary in the hierarchy (i.e. a summary of a summary)

In [ ]:
print("Top Cosine Similarity Scores: ", get_top_similarities(cosine_similarities))
show_related_docs(document, corpus, related_docs_indices)

print("\n" + "SUMMARIZATION RESULTS" + "\n")
print(divString(100) + "\n")
print("EXTRACTIVE SUMMARY PRIMARY:" + "\n", extractive_summary_primary)
print("\n")
print("ABSTRACTIVE SUMMARY PRIMARY:" + "\n", abstractive_summary_primary)
print("\n")
print("ABSTRACTIVE SUMMARY SECONDARY:" + "\n", abstractive_summary_secondary)
print(divString(100) + "\n")

In [ ]:
######################
# ITERATIVE PIPELINE #
######################

# Initial Document Selection
search_results_df = search_and_subset_data(covid_test_data, "death") # a df containing entries that have death in the first paragraph
corpus = search_results_df.first_paragraph.to_list()  # Converting to a list to vectorize the entries
initial_document = select_random_document(search_results_df).first_paragraph.values[0] # Selected a random row from the search results dataframe and extracted the first paragraph text to serve as our document

# Pipeline Iteration
iterate_pipeline(initial_document, corpus, num_iter = 5, aggregate_doc_variant = "abstractive_secondary")

# Scratch Code

In [ ]:
search_results_df = search_and_subset_data(covid_test_data, "death") # a df containing entries that have death in the first paragraph
search_results = search_results_df.first_paragraph.to_list()  # Converting to a list to vectorize the entries
corpus = vectorizer.fit_transform(search_results) # Vectorize and fit transform
document = select_random_document(x).first_paragraph.values[0] # Selected a random row from the search results dataframe and extracted the first paragraph text to serve as our document
# compute_cosine_similarities(document, corpus)

In [ ]:
select_random_document(x).first_paragraph.values[0]

In [ ]:
x = search_and_subset_data(covid_test_data, "death") # a df containing entries that have death in the first paragraph
y = x.first_paragraph.to_list() # Converting to a list to vectorize the entries
z = vectorizer.fit_transform(y) # Vectorize and fit transform

cosine_similarities = linear_kernel(z[0], z).flatten() #Compute the similarities between a single vectorized article and the rest of the vectorized corpus
cosine_similarities